# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing libraries

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
import re
import gensim
from gensim.corpora import WikiCorpus
from gensim.models import KeyedVectors
import smart_open
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
!pip install contractions
import contractions
nltk.download('universal_tagset')
from scipy.spatial import distance

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
import pandas as pd
import glob
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import joblib
import numpy as np
import requests
import bs4

# Preprocessing

In [ ]:
def cleaning(text):
  line = contractions.fix(text)
  line = re.sub(r'\'(\w|\s)','',line)
  line = re.sub(r'(\.{2,}|\,|\?+|!+|\:|\-\-|\'\'|``|\(|\))','',line)
  line = re.sub(r'[`\!@#$%^&\*()\_+={}\:\;<>\?\|\-\.]',' ', line)
  line = line.lower()
  return line

In [ ]:
def tokenization(text):
  text_list = word_tokenize(text)
  return text_list;

In [ ]:
def removeStopWords(text_list):
  llist=[]
  stopwordsList = stopwords.words('english')
  for word in text_list:
    if word not in stopwordsList:
       llist.append(word)
  return llist     

In [ ]:
def normalization(text_list):
  llist=[]
  lemmatizer = WordNetLemmatizer() 
  for word in text_list:
       llist.append(lemmatizer.lemmatize(word))
  return llist

Lemmatize+contractions+tokeniztion+cleaning


In [ ]:
def preprocessing(file,flag):
  if(flag==1):
    file_name = open(file,'r')
    text = cleaning(file_name.read())
    llist = tokenization(text)
    llist = removeStopWords(llist)
    llist = normalization(llist)
    return llist
      
  elif(flag==0):
       text = cleaning(file)
       llist = tokenization(text)
       llist = removeStopWords(llist)
       llist = normalization(llist)
       return llist

In [ ]:
def getPosTag(words):
    #Getting POS Tag
    tagList = nltk.pos_tag(words,  tagset='universal')
    #Format - WORD_POSTAG
    for i in range(len(words)):
        words[i] = words[i] + '_' + tagList[i][1]
    return words

# Word2Vec for *words*

In [ ]:
def getWord2Vec(fileid):
    words = preprocessing(fileid,0)
    words = getPosTag(words)
    model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/IR/enwiki_dbow/model.txt', binary=False, unicode_errors='ignore') 
    #model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Sem2/IR/Project/Models/Doc2Vec/enwiki_dbow/model.txt', binary=False, unicode_errors='ignore')  
    combinedVector = [0]*300
    for word in words:
      if word in model: 
        vector = model[word]
        for i in range(len(vector)):
          combinedVector[i] += vector[i] 
        for i in range(len(combinedVector)):
          combinedVector[i] = combinedVector[i]/ len(words)
    return combinedVector

# Load Model

In [ ]:
def loadmodelD2V():
  model = gensim.models.Doc2Vec.load('/content/drive/MyDrive/IR/enwiki_dbow/doc2vec.bin')
  return model

# Doc2Vec 

In [ ]:
# doc id input , vector output
def getDoc2Vec(sents,model):
  para_list = sents.split('\n\n')
  Vec_list = []
  for para in para_list:
    test_data = preprocessing(para,0)
    vec = model.infer_vector(test_data)
    Vec_list.append(vec)
  print(len(Vec_list))     
  return Vec_list 

# Cosine Similarity

In [ ]:
# def normalize(list1):
#   list2 = [0]*len(list1)
#   amin, amax = min(list1), max(list1)
#   for i, val in enumerate(list1):
#     list2[i] = (val-amin) / (amax-amin)
#   return list2

def cosine_similarity(set1,set2):
  X_set = set1
  Y_set = set2
  # X_set = normalize(X_set)
  # Y_set = normalize(Y_set)
  c = 0
  l1 = X_set
  l2 = Y_set
  rvector = len(l1)
  # cosine formula  
  for i in range(rvector): 
    c+= l1[i]*l2[i] 
  cosine = c / float(  ( sum(l1)*sum(l2) )**0.5  ) 
  return cosine

def ranking(X,Y):
  X = X.split('\n\n') 
  score_for_each_para = Y
  return [x for _,x in sorted(zip(score_for_each_para,X))]

# Testing

In [ ]:
doc = '''A car (or automobile) is a wheeled motor vehicle used for transportation. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four wheels, and mainly transport people rather than goods.[2][3]\n\nCars came into global use during the 20th century, and developed economies depend on them. The year 1886 is regarded as the birth year of the modern car when German inventor Karl Benz patented his Benz Patent-Motorwagen. Cars became widely available in the early 20th century. One of the first cars accessible to the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced animal-drawn carriages and carts, but took much longer to be accepted in Western Europe and other parts of the world.[citation needed]\n\nCars have controls for driving, parking, passenger comfort, and a variety of lights. Over the decades, additional features and controls have been added to vehicles, making them progressively more complex, but also more reliable and easier to operate.[citation needed] These include rear-reversing cameras, air conditioning, navigation systems, and in-car entertainment. Most cars in use in the 2010s are propelled by an internal combustion engine, fueled by the combustion of fossil fuels. Electric cars, which were invented early in the history of the car, became commercially available in the 2000s and are predicted to cost less to buy than gasoline cars before 2025.[4][5] The transition from fossil fuels to electric cars features prominently in most climate change mitigation scenarios,[6] such as Project Drawdown\'s 100 actionable solutions for climate change.[7]\n\nThere are costs and benefits to car use. The costs to the individual include acquiring the vehicle, interest payments (if the car is financed), repairs and maintenance, fuel, depreciation, driving time, parking fees, taxes, and insurance.[8] The costs to society include maintaining roads, land use, road congestion, air pollution, public health, healthcare, and disposing of the vehicle at the end of its life. Traffic collisions are the largest cause of injury-related deaths worldwide.[9]\n\nThe personal benefits include on-demand transportation, mobility, independence, and convenience.[10] The societal benefits include economic benefits, such as job and wealth creation from the automotive industry, transportation provision, societal well-being from leisure and travel opportunities, and revenue generation from the taxes. People\'s ability to move flexibly from place to place has far-reaching implications for the nature of societies.[11] There are around 1 billion cars in use worldwide. The numbers are increasing rapidly, especially in China, India and other newly industrialized countries.[12]\n\nThe English word car is believed to originate from Latin carrus/carrum "wheeled vehicle" or (via Old North French) Middle English carre "two-wheeled cart," both of which in turn derive from Gaulish karros "chariot."[13][14] It originally referred to any wheeled horse-drawn vehicle, such as a cart, carriage, or wagon.[15][16]\n\n"Motor car," attested from 1895, is the usual formal term in British English.[3] "Autocar," a variant likewise attested from 1895 and literally meaning "self-propelled car," is now considered archaic.[17] "Horseless carriage" is attested from 1895.[18]\n\n"Automobile," a classical compound derived from Ancient Greek autós (αὐτός) "self" and Latin mobilis "movable," entered English from French and was first adopted by the Automobile Club of Great Britain in 1897.[19] It fell out of favour in Britain and is now used chiefly in North America,[20] where the abbreviated form "auto" commonly appears as an adjective in compound formations like "auto industry" and "auto mechanic".[21][22] Both forms are still used in everyday Dutch (auto/automobiel) and German (Auto/Automobil).[citation needed]\n\nThe first working steam-powered vehicle was designed—and quite possibly built—by Ferdinand Verbiest, a Flemish member of a Jesuit mission in China around 1672. It was a 65-centimetre (26\xa0in)-long scale-model toy for the Kangxi Emperor that was unable to carry a driver or a passenger.[10][23][24] It is not known with certainty if Verbiest\'s model was successfully built or run.[24]\n\nNicolas-Joseph Cugnot is widely credited with building the first full-scale, self-propelled mechanical vehicle or car in about 1769; he created a steam-powered tricycle.[25]  He also constructed two steam tractors for the French Army, one of which is preserved in the French National Conservatory of Arts and Crafts.[26] His inventions were, however, handicapped by problems with water supply and maintaining steam pressure.[26] In 1801, Richard Trevithick built and demonstrated his Puffing Devil road locomotive, believed by many to be the first demonstration of a steam-powered road vehicle. It was unable to maintain sufficient steam pressure for long periods and was of little practical use.\n\nThe development of external combustion engines is detailed as part of the history of the car but often treated separately from the development of true cars.  A variety of steam-powered road vehicles were used during the first part of the 19th century, including steam cars, steam buses, phaetons, and steam rollers.  Sentiment against them led to the Locomotive Acts of 1865.\n\nIn 1807, Nicéphore Niépce and his brother Claude created what was probably the world\'s first internal combustion engine (which they called a Pyréolophore), but they chose to install it in a boat on the river Saone in France.[27] Coincidentally, in 1807 the Swiss inventor François Isaac de Rivaz designed his own \'de Rivaz internal combustion engine\' and used it to develop the world\'s first vehicle to be powered by such an engine. The Niépces\' Pyréolophore was fuelled by a mixture of Lycopodium powder (dried spores of the Lycopodium plant), finely crushed coal dust and resin that were mixed with oil, whereas de Rivaz used a mixture of hydrogen and oxygen.[27] Neither design was very successful, as was the case with others, such as Samuel Brown, Samuel Morey, and Etienne Lenoir with his hippomobile, who each produced vehicles (usually adapted carriages or carts) powered by internal combustion engines.[1]\n\nIn November 1881, French inventor Gustave Trouvé demonstrated the first working (three-wheeled) car powered by electricity at the International Exposition of Electricity, Paris.[28] Although several other German engineers (including Gottlieb Daimler, Wilhelm Maybach, and Siegfried Marcus) were working on the problem at about the same time, Karl Benz generally is acknowledged as the inventor of the modern car.[1]\n\nIn 1879, Benz was granted a patent for his first engine, which had been designed in 1878. Many of his other inventions made the use of the internal combustion engine feasible for powering a vehicle. His first Motorwagen was built in 1885 in Mannheim, Germany. He was awarded the patent for its invention as of his application on 29 January 1886 (under the auspices of his major company, Benz & Cie., which was founded in 1883). Benz began promotion of the vehicle on 3 July 1886, and about 25 Benz vehicles were sold between 1888 and 1893, when his first four-wheeler was introduced along with a cheaper model. They also were powered with four-stroke engines of his own design. Emile Roger of France, already producing Benz engines under license, now added the Benz car to his line of products. Because France was more open to the early cars, initially more were built and sold in France through Roger than Benz sold in Germany. In August 1888 Bertha Benz, the wife of Karl Benz, undertook the first road trip by car, to prove the road-worthiness of her husband\'s invention.\n\nIn 1896, Benz designed and patented the first internal-combustion flat engine, called boxermotor. During the last years of the nineteenth century, Benz was the largest car company in the world with 572 units produced in 1899 and, because of its size, Benz & Cie., became a joint-stock company. The first motor car in central Europe and one of the first factory-made cars in the world, was produced by Czech company Nesselsdorfer Wagenbau (later renamed to Tatra) in 1897, the Präsident automobil.\n\nDaimler and Maybach founded Daimler Motoren Gesellschaft (DMG) in Cannstatt in 1890, and sold their first car in 1892 under the brand name Daimler. It was a horse-drawn stagecoach built by another manufacturer, which they retrofitted with an engine of their design. By 1895 about 30 vehicles had been built by Daimler and Maybach, either at the Daimler works or in the Hotel Hermann, where they set up shop after disputes with their backers. Benz, Maybach and the Daimler team seem to have been unaware of each other\'s early work. They never worked together; by the time of the merger of the two companies, Daimler and Maybach were no longer part of DMG. Daimler died in 1900 and later that year, Maybach designed an engine named Daimler-Mercedes that was placed in a specially ordered model built to specifications set by Emil Jellinek. This was a production of a small number of vehicles for Jellinek to race and market in his country. Two years later, in 1902, a new model DMG car was produced and the model was named Mercedes after the Maybach engine, which generated 35\xa0hp. Maybach quit DMG shortly thereafter and opened a business of his own. Rights to the Daimler brand name were sold to other manufacturers.\n\nKarl Benz proposed co-operation between DMG and Benz & Cie. when economic conditions began to deteriorate in Germany following the First World War, but the directors of DMG refused to consider it initially. Negotiations between the two companies resumed several years later when these conditions worsened, and in 1924, they signed an Agreement of Mutual Interest, valid until the year 2000. Both enterprises standardized design, production, purchasing, and sales and they advertised or marketed their car models jointly, although keeping their respective brands. On 28 June 1926, Benz & Cie. and DMG finally merged as the Daimler-Benz company, baptizing all of its cars Mercedes Benz, as a brand honoring the most important model of the DMG cars, the Maybach design later referred to as the 1902 Mercedes-35\xa0hp, along with the Benz name. Karl Benz remained a member of the board of directors of Daimler-Benz until his death in 1929, and at times, his two sons also participated in the management of the company.\n\nIn 1890, Émile Levassor and Armand Peugeot of France began producing vehicles with Daimler engines, and so laid the foundation of the automotive industry in France. In 1891, Auguste Doriot and his Peugeot colleague Louis Rigoulot completed the longest trip by a gasoline-powered vehicle when their self-designed and built Daimler powered Peugeot Type 3 completed 2,100\xa0km (1,300 miles) from Valentigney to Paris and Brest and back again. They were attached to the first Paris–Brest–Paris bicycle race, but finished 6 days after the winning cyclist, Charles Terront.\n\nThe first design for an American car with a gasoline internal combustion engine was made in 1877 by George Selden of Rochester, New York. Selden applied for a patent for a car in 1879, but the patent application expired because the vehicle was never built. After a delay of sixteen years and a series of attachments to his application, on 5 November 1895, Selden was granted a United States patent (U.S. Patent 549,160) for a two-stroke car engine, which hindered, more than encouraged, development of cars in the United States. His patent was challenged by Henry Ford and others, and overturned in 1911.\n\nIn 1893, the first running, gasoline-powered American car was built and road-tested by the Duryea brothers of Springfield, Massachusetts. The first public run of the Duryea Motor Wagon took place on 21 September 1893, on Taylor Street in Metro Center Springfield.[29][30] The Studebaker Automobile Company, subsidiary of a long-established wagon and coach manufacturer, started to build cars in 1897[31]:p.66 and commenced sales of electric vehicles in 1902 and gasoline vehicles in 1904.[32]\n\nIn Britain, there had been several attempts to build steam cars with varying degrees of success, with Thomas Rickett even attempting a production run in 1860.[33] Santler from Malvern is recognized by the Veteran Car Club of Great Britain as having made the first gasoline-powered car in the country in 1894,[34] followed by Frederick William Lanchester in 1895, but these were both one-offs.[34] The first production vehicles in Great Britain came from the Daimler Company, a company founded by Harry J. Lawson in 1896, after purchasing the right to use the name of the engines. Lawson\'s company made its first car in 1897, and they bore the name Daimler.[34]\n\nIn 1892, German engineer Rudolf Diesel was granted a patent for a "New Rational Combustion Engine". In 1897, he built the first diesel engine.[1] Steam-, electric-, and gasoline-powered vehicles competed for decades, with gasoline internal combustion engines achieving dominance in the 1910s. Although various pistonless rotary engine designs have attempted to compete with the conventional piston and crankshaft design, only Mazda\'s version of the Wankel engine has had more than very limited success.\n\nAll in all, it is estimated that over 100,000 patents created the modern automobile and motorcycle.[35]\n\nLarge-scale, production-line manufacturing of affordable cars was started by Ransom Olds in 1901 at his Oldsmobile factory in Lansing, Michigan and based upon stationary assembly line techniques pioneered by Marc Isambard Brunel at the Portsmouth Block Mills, England, in 1802. The assembly line style of mass production and interchangeable parts had been pioneered in the U.S. by Thomas Blanchard in 1821, at the Springfield Armory in Springfield, Massachusetts.[36] This concept was greatly expanded by Henry Ford, beginning in 1913 with the world\'s first moving assembly line for cars at the Highland Park Ford Plant.\n\nAs a result, Ford\'s cars came off the line in fifteen-minute intervals, much faster than previous methods, increasing productivity eightfold, while using less manpower (from 12.5-man-hours to 1 hour 33 minutes).[37] It was so successful, paint became a bottleneck. Only Japan black would dry fast enough, forcing the company to drop the variety of colors available before 1913, until fast-drying Duco lacquer was developed in 1926. This is the source of Ford\'s apocryphal remark, "any color as long as it\'s black".[37] In 1914, an assembly line worker could buy a Model T with four months\' pay.[37]\n\nFord\'s complex safety procedures—especially assigning each worker to a specific location instead of allowing them to roam about—dramatically reduced the rate of injury.[citation needed] The combination of high wages and high efficiency is called "Fordism," and was copied by most major industries. The efficiency gains from the assembly line also coincided with the economic rise of the United States. The assembly line forced workers to work at a certain pace with very repetitive motions which led to more output per worker while other countries were using less productive methods.\n\nIn the automotive industry, its success was dominating, and quickly spread worldwide seeing the founding of Ford France and Ford Britain in 1911, Ford Denmark 1923, Ford Germany 1925; in 1921, Citroen was the first native European manufacturer to adopt the production method. Soon, companies had to have assembly lines, or risk going broke; by 1930, 250 companies which did not, had disappeared.[37]\n\nDevelopment of automotive technology was rapid, due in part to the hundreds of small manufacturers competing to gain the world\'s attention. Key developments included electric ignition and the electric self-starter (both by Charles Kettering, for the Cadillac Motor Company in 1910–1911), independent suspension, and four-wheel brakes.\n\nSince the 1920s, nearly all cars have been mass-produced to meet market needs, so marketing plans often have heavily influenced car design. It was Alfred P. Sloan who established the idea of different makes of cars produced by one company, called the General Motors Companion Make Program, so that buyers could "move up" as their fortunes improved.\n\nReflecting the rapid pace of change, makes shared parts with one another so larger production volume resulted in lower costs for each price range. For example, in the 1930s, LaSalles, sold by Cadillac, used cheaper mechanical parts made by Oldsmobile; in the 1950s, Chevrolet shared hood, doors, roof, and windows with Pontiac; by the 1990s, corporate powertrains and shared platforms (with interchangeable brakes, suspension, and other parts) were common. Even so, only major makers could afford high costs, and even companies with decades of production, such as Apperson, Cole, Dorris, Haynes, or Premier, could not manage: of some two hundred American car makers in existence in 1920, only 43 survived in 1930, and with the Great Depression, by 1940, only 17 of those were left.[37]\n\nIn Europe, much the same would happen. Morris set up its production line at Cowley in 1924, and soon outsold Ford, while beginning in 1923 to follow Ford\'s practice of vertical integration, buying Hotchkiss (engines), Wrigley (gearboxes), and Osberton (radiators), for instance, as well as competitors, such as Wolseley: in 1925, Morris had 41% of total British car production. Most British small-car assemblers, from Abbey to Xtra, had gone under. Citroen did the same in France, coming to cars in 1919; between them and other cheap cars in reply such as Renault\'s 10CV and Peugeot\'s 5CV, they produced 550,000 cars in 1925, and Mors, Hurtu, and others could not compete.[37] Germany\'s first mass-manufactured car, the Opel 4PS Laubfrosch (Tree Frog), came off the line at Russelsheim in 1924, soon making Opel the top car builder in Germany, with 37.5% of the market.[37]\n\nIn Japan, car production was very limited before World War II. Only a handful of companies were producing vehicles in limited numbers, and these were small, three-wheeled for commercial uses, like Daihatsu, or were the result of partnering with European companies, like Isuzu building the Wolseley A-9 in 1922. Mitsubishi was also partnered with Fiat and built the Mitsubishi Model A based on a Fiat vehicle. Toyota, Nissan, Suzuki, Mazda, and Honda began as companies producing non-automotive products before the war, switching to car production during the 1950s. Kiichiro Toyoda\'s decision to take Toyoda Loom Works into automobile manufacturing would create what would eventually become Toyota Motor Corporation, the largest automobile manufacturer in the world. Subaru, meanwhile, was formed from a conglomerate of six companies who banded together as Fuji Heavy Industries, as a result of having been broken up under keiretsu legislation.\n\nAccording to the European Environment Agency, the transport sector is a major contributor to air pollution, noise pollution and climate change.[38]\n\nMost cars in use in the 2010s run on gasoline burnt in an internal combustion engine (ICE). The International Organization of Motor Vehicle Manufacturers says that, in countries that mandate low sulfur gasoline, gasoline-fuelled cars built to late 2010s standards (such as Euro-6) emit very little local air pollution.[39][40]  Some cities ban older gasoline-fuelled cars and some countries plan to ban sales in future. However some environmental groups say this phase-out of fossil fuel vehicles must be brought forward to limit climate change. Production of gasoline fueled cars peaked in 2017.[41][42]\n\nOther hydrocarbon fossil fuels also burnt by deflagration (rather than detonation) in ICE cars include diesel, Autogas and CNG. Removal of fossil fuel subsidies,[43][44] concerns about oil dependence, tightening environmental laws and restrictions on greenhouse gas emissions are propelling work on alternative power systems for cars. This includes hybrid vehicles, plug-in electric vehicles and hydrogen vehicles. 2.1 million light electric vehicles (of all types but mainly cars) were sold in 2018, over half in China: this was an increase of 64% on the previous year, giving a global total on the road of 5.4 million.[45]  Vehicles using alternative fuels such as ethanol flexible-fuel vehicles and natural gas vehicles[clarification needed] are also gaining popularity in some countries.[citation needed] Cars for racing or speed records have sometimes employed jet or rocket engines, but these are impractical for common use.\n\nOil consumption has increased rapidly in the 20th and 21st centuries because there are more cars; the 1985–2003 oil glut even fuelled the sales of low-economy vehicles in OECD countries. The BRIC countries are adding to this consumption.\n\nCars are equipped with controls used for driving, passenger comfort, and safety, normally operated by a combination of the use of feet and hands, and occasionally by voice on 21st-century cars. These controls include a steering wheel, pedals for operating the brakes and controlling the car\'s speed (and, in a manual transmission car, a clutch pedal), a shift lever or stick for changing gears, and a number of buttons and dials for turning on lights, ventilation, and other functions. Modern cars\' controls are now standardized, such as the location for the accelerator and brake, but this was not always the case. Controls are evolving in response to new technologies, for example, the electric car and the integration of mobile communications.\n\nSome of the original controls are no longer required. For example, all cars once had controls for the choke valve, clutch, ignition timing, and a crank instead of an electric starter. However new controls have also been added to vehicles, making them more complex. These include air conditioning, navigation systems, and in car entertainment. Another trend is the replacement of physical knobs and switches by secondary controls with touchscreen controls such as BMW\'s iDrive and Ford\'s MyFord Touch. Another change is that while early cars\' pedals were physically linked to the brake mechanism and throttle, in the 2010s, cars have increasingly replaced these physical linkages with electronic controls.\n\nCars are typically fitted with multiple types of lights. These include headlights, which are used to illuminate the way ahead and make the car visible to other users, so that the vehicle can be used at night; in some jurisdictions, daytime running lights; red brake lights to indicate when the brakes are applied; amber turn signal lights to indicate the turn intentions of the driver; white-colored reverse lights to illuminate the area behind the car (and indicate that the driver will be or is reversing); and on some vehicles, additional lights (e.g., side marker lights) to increase the visibility of the car. Interior lights on the ceiling of the car are usually fitted for the driver and passengers. Some vehicles also have a trunk light and, more rarely, an engine compartment light.\n\nDuring the late 20th and early 21st century cars increased in weight due to batteries,[47] modern steel safety cages, anti-lock brakes, airbags, and "more-powerful—if more-efficient—engines"[48] and, as of 2019[update], typically weigh between 1 and 3 tonnes.[49] Heavier cars are safer for the driver from a crash perspective, but more dangerous for other vehicles and road users.[48] The weight of a car influences fuel consumption and performance, with more weight resulting in increased fuel consumption and decreased performance. The SmartFortwo, a small city car, weighs 750–795\xa0kg (1,655–1,755\xa0lb). Heavier cars include full-size cars, SUVs and extended-length SUVs like the Suburban.\n\nAccording to research conducted by Julian Allwood of the University of Cambridge, global energy use could be greatly reduced by using lighter cars, and an average weight of 500\xa0kg (1,100\xa0lb) has been said to be well achievable.[50][better\xa0source\xa0needed] In some competitions such as the Shell Eco Marathon, average car weights of 45\xa0kg (99\xa0lb) have also been achieved.[51] These cars are only single-seaters (still falling within the definition of a car, although 4-seater cars are more common), but they nevertheless demonstrate the amount by which car weights could still be reduced, and the subsequent lower fuel use (i.e. up to a fuel use of 2560\xa0km/l).[52]\n\nMost cars are designed to carry multiple occupants, often with four or five seats. Cars with five seats typically seat two passengers in the front and three in the rear. Full-size cars and large sport utility vehicles can often carry six, seven, or more occupants depending on the arrangement of the seats. On the other hand, sports cars are most often designed with only two seats. The differing needs for passenger capacity and their luggage or cargo space has resulted in the availability of a large variety of body styles to meet individual consumer requirements that include, among others, the sedan/saloon, hatchback, station wagon/estate, and minivan.\n\nTraffic collisions are the largest cause of injury-related deaths worldwide.[9] Mary Ward became one of the first documented car fatalities in 1869 in Parsonstown, Ireland,[53] and Henry Bliss one of the United States\' first pedestrian car casualties in 1899 in New York City.[54]\nThere are now standard tests for safety in new cars, such as the EuroNCAP and the US NCAP tests,[55] and insurance-industry-backed tests by the Insurance Institute for Highway Safety (IIHS).[56]\n\nThe costs of car usage, which may include the cost of: acquiring the vehicle, repairs and auto maintenance, fuel, depreciation, driving time, parking fees, taxes, and insurance,[8] are weighed against the cost of the alternatives, and the value of the benefits\xa0– perceived and real\xa0– of vehicle usage. The benefits may include on-demand transportation, mobility, independence and convenience.[10] During the 1920s, cars had another benefit: "[c]ouples finally had a way to head off on unchaperoned dates, plus they had a private space to snuggle up close at the end of the night."[58]\n\nSimilarly the costs to society of car use may include; maintaining roads, land use, air pollution, road congestion, public health, health care, and of disposing of the vehicle at the end of its life; and can be balanced against the value of the benefits to society that car use generates. Societal benefits may include: economy benefits, such as job and wealth creation, of car production and maintenance, transportation provision, society wellbeing derived from leisure and travel opportunities, and revenue generation from the tax opportunities. The ability of humans to move flexibly from place to place has far-reaching implications for the nature of societies.[11]\n\nCars are a major cause of urban air pollution,[59] with all types of cars producing dust from brakes, tyres and road wear.[60]  As of 2018[update] the average diesel car has a worse effect on air quality than the average gasoline car[61] But both gasoline and diesel cars pollute more than electric cars.[62] While there are different ways to power cars most rely on gasoline or diesel, and they consume almost a quarter of world oil production as of 2019[update].[41] In 2018 passenger road vehicles emitted 3.6 gigatonnes of carbon dioxide.[63] As of 2019[update], due to greenhouse gases emitted during battery production, electric cars must be driven tens of thousands of kilometers before their lifecycle carbon emissions are less than fossil fuel cars:[64] but this is expected to improve in future due to longer lasting[65] batteries being produced in larger factories,[66] and lower carbon electricity. Many governments are using fiscal policies, such as road tax, to discourage the purchase and use of more polluting cars;[67] and many cities are doing the same with low-emission zones.[68] Fuel taxes may act as an incentive for the production of more efficient, hence less polluting, car designs (e.g. hybrid vehicles) and the development of alternative fuels. High fuel taxes or cultural change may provide a strong incentive for consumers to purchase lighter, smaller, more fuel-efficient cars, or to not drive.[68]\n\nThe lifetime of a car built in the 2020s is expected to be about 16 years, or about 2 million kilometres (1.2 million miles) if driven a lot.[69] According to the International Energy Agency fuel economy improved 0.7% in 2017, but an annual improvement of 3.7% is needed to meet the Global Fuel Economy Initiative 2030 target.[70] The increase in sales of SUVs is bad for fuel economy.[41] Many cities in Europe, have banned older fossil fuel cars and all fossil fuel vehicles will be banned in Amsterdam from 2030.[71] Many Chinese cities limit licensing of fossil fuel cars,[72] and many countries plan to stop selling them between 2025 and 2050.[73]\n\nThe manufacture of vehicles is resource intensive, and many manufacturers now report on the environmental performance of their factories, including energy usage, waste and water consumption.[74] Manufacturing each kWh of battery emits a similar amount of carbon as burning through one full tank of gasoline.[75] The growth in popularity of the car allowed cities to sprawl, therefore encouraging more travel by car resulting in inactivity and obesity, which in turn can lead to increased risk of a variety of diseases.[76]\n\nAnimals and plants are often negatively impacted by cars via habitat destruction and pollution. Over the lifetime of the average car the "loss of habitat potential" may be over 50,000\xa0m2 (540,000\xa0sq\xa0ft) based on primary production correlations.[77] Animals are also killed every year on roads by cars, referred to as roadkill. More recent road developments are including significant environmental mitigation in their designs, such as green bridges (designed to allow wildlife crossings) and creating wildlife corridors.\n\nGrowth in the popularity of vehicles and commuting has led to traffic congestion. Moscow, Istanbul, Bogota, Mexico City and Sao Paulo were the world\'s most congested cities in 2018 according to INRIX, a data analytics company.[78]\n\nAlthough intensive development of  conventional battery electric vehicles is continuing into the 2020s,[79] other car propulsion technologies that are under development include wheel hub motors,[80] wireless charging,[81] hydrogen cars,[82] and hydrogen/electric hybrids.[83] Research into alternative forms of power includes using ammonia instead of hydrogen in fuel cells.[84]\n\nNew materials[85] which may replace steel car bodies include duralumin, fiberglass, carbon fiber, biocomposites, and carbon nanotubes. Telematics technology is allowing more and more people to share cars, on a pay-as-you-go basis, through car share and carpool schemes. Communication is also evolving due to connected car systems.[86]\n\nFully autonomous vehicles, also known as driverless cars, already exist in prototype (such as the Google driverless car), but have a long way to go before they are in general use.\n\nThere have been several projects aiming to develop a car on the principles of open design, an approach to designing in which the plans for the machinery and systems are publicly shared, often without monetary compensation. The projects include OScar, Riversimple (through 40fires.org)[87] and c,mm,n.[88] None of the projects have reached significant success in terms of developing a car as a whole both from hardware and software perspective and no mass production ready open-source based design have been introduced as of late 2009. Some car hacking through on-board diagnostics (OBD) has been done so far.[89]\n\nCar-share arrangements and carpooling are also increasingly popular, in the US and Europe.[90] For example, in the US, some car-sharing services have experienced double-digit growth in revenue and membership growth between 2006 and 2007. Services like car sharing offering a residents to "share" a vehicle rather than own a car in already congested neighborhoods.[91]\n\nThe automotive industry designs, develops, manufactures, markets, and sells the world\'s motor vehicles, more than three-quarters of which are cars. In 2018 there were 70 million cars manufactured worldwide,[92] down 2 million from the previous year.[93]\n\nThe automotive industry in China produces by far the most (24 million in 2018), followed by Japan (8 million), Germany (5 million) and India (4 million).[92] The largest market is China, followed by the USA.\n\nAround the world there are about a billion cars on the road;[94] they burn over a trillion liters of gasoline and diesel fuel yearly, consuming about 50 EJ (nearly 300 terawatt-hours) of energy.[95] The numbers of cars are increasing rapidly in China and India.[12] In the opinion of some, urban transport systems based around the car have proved unsustainable, consuming excessive energy, affecting the health of populations, and delivering a declining level of service despite increasing investment. Many of these negative impacts fall disproportionately on those social groups who are also least likely to own and drive cars.[96][97] The sustainable transport movement focuses on solutions to these problems. The car industry is also facing increasing competition from the public transport sector, as some people re-evaluate their private vehicle usage.\n\nEstablished alternatives for some aspects of car use include public transport such as buses, trolleybuses, trains, subways, tramways, light rail, cycling, and walking. Bicycle sharing systems have been established in China and many European cities, including Copenhagen and Amsterdam. Similar programs have been developed in large US cities.[99][100] Additional individual modes of transport, such as personal rapid transit could serve as an alternative to cars if they prove to be socially accepted.[101]\n\nThe term motorcar was formerly also used in the context of electrified rail systems to denote a car which functions as a small locomotive but also provides space for passengers and baggage.  These locomotive cars were often used on suburban routes by both interurban and intercity railroad systems.[102]\n'''

query = "A car (or automobile) is a wheeled motor vehicle used for transportation. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four wheels, and mainly transport people rather than goods.[2][3]"

In [ ]:
doc='''Every item on this page was chosen by a Town & Country editor. We may earn commission on some of the items you choose to buy.\n\nI am my mother’s son. And I am our son’s mother. Together we bring you Archewell," the couple wrote on the homepage for their new non-profit.\n\nJust ahead of the new year, Prince Harry and Meghan Markle have updated the website for their new non-profit, Archewell. The site was launched in October with just a landing page, but now, the couple has fleshed it out, adding pages for the Archewell Foundation, Archewell Audio, and Archewell Productions.\n\nThe Duke and Duchess of Sussex also added "A Letter for 2021" to the homepage, along with black-and-white childhood photos of Harry and Meghan with their mothers, writing "I am my mother’s son. And I am our son’s mother. Together we bring you Archewell." The photo of Harry appears to be one taken while he was sitting on Princess Diana\'s shoulders at Highgrove in 1986.\n\nIn the short message, they ask the public to join them as they "work to build a better world, one act of compassion at a time." At the bottom of the homepage, there\'s also a place for visitors to submit their stories "about how you or somebody you know acted with compassion in the last year," or to "describe when you felt connected with your friends, family, or community, despite the distance.\n\nOn the Archewell Foundation page, Harry and Meghan have listed a series of partnerships and projects the non-profit is undertaking, including those with James R. Doty's Center for Compassion and Altruism Research and Education (CCARE), Tristan Harris' Center for Humane Technology, Rachel Cargle's Loveland Foundation, Dr. Safiya Noble and Dr. Sarah T. Roberts' UCLA Center for Critical Internet Inquiry, and José Andrés' World Central Kitchen.\n\nThe Archewell Audio page features the first episode of Archewell Audio, Harry and Meghan\'s new podcast with Spotify, as well as a short description of the production company\'s aims: to "produce programming that uplifts and entertains audiences around the world," and "spotlight diverse perspectives and voices," as well as "build community through shared experiences, powerful narratives, and universal values.\n\nAnd finally, the Archewell Productions page contains a description of the couple\'s newly-creation video production company, which was launched after Harry and Meghan inked a landmark deal with Netflix earlier this year. "Archewell Productions will utilize the power of storytelling to embrace our shared humanity and duty to truth through a compassionate lens," the page reads.\n\nArchewell press secretary Toya Holness told Town & Country, “Founded earlier this year by The Duke and Duchess of Sussex, Archewell uplifts communities through non-profit partnerships and creative activations. It’s a place where compassion matters, communities gather, and storytelling is the engine. The website has been updated to reflect the work Archewell has undertaken throughout 2020 and to create a place for people and communities around the world to share their stories.\n\nPeruse the website here—and read the Duke and Duchess\'s full "Letter for 2021" below:  '''

In [ ]:
model = loadmodelD2V()
v1 = getDoc2Vec(doc,model)
v2 = getWord2Vec(query)

10


In [ ]:
scores= []
for i in range(len(v1)):
  scores.append(distance.cosine(v1[i],v2))
rank = ranking(doc,scores)
for i in range(len(rank)):
  print(i,"::::"+rank[i])

0 ::::Archewell press secretary Toya Holness told Town & Country, “Founded earlier this year by The Duke and Duchess of Sussex, Archewell uplifts communities through non-profit partnerships and creative activations. It’s a place where compassion matters, communities gather, and storytelling is the engine. The website has been updated to reflect the work Archewell has undertaken throughout 2020 and to create a place for people and communities around the world to share their stories.
1 ::::On the Archewell Foundation page, Harry and Meghan have listed a series of partnerships and projects the non-profit is undertaking, including those with James R. Doty's Center for Compassion and Altruism Research and Education (CCARE), Tristan Harris' Center for Humane Technology, Rachel Cargle's Loveland Foundation, Dr. Safiya Noble and Dr. Sarah T. Roberts' UCLA Center for Critical Internet Inquiry, and José Andrés' World Central Kitchen.
2 ::::And finally, the Archewell Productions page contains a d

In [ ]:
def wmdist(doc,doc_list,query):
    scores = []
    for i in range(len(doc_list)):
      scores.append(model.wmdistance(doc_list[i],query))
    rank = ranking(doc,scores)
    return rank

In [ ]:
#query = cleaning(query)
#doc = cleaning(doc)
doc_list = doc.split('\n\n')
most_relevant_paragraph = wmdist(doc,doc_list,query)
print(most_relevant_paragraph)

["On the Archewell Foundation page, Harry and Meghan have listed a series of partnerships and projects the non-profit is undertaking, including those with James R. Doty's Center for Compassion and Altruism Research and Education (CCARE), Tristan Harris' Center for Humane Technology, Rachel Cargle's Loveland Foundation, Dr. Safiya Noble and Dr. Sarah T. Roberts' UCLA Center for Critical Internet Inquiry, and José Andrés' World Central Kitchen.", 'And finally, the Archewell Productions page contains a description of the couple\'s newly-creation video production company, which was launched after Harry and Meghan inked a landmark deal with Netflix earlier this year. "Archewell Productions will utilize the power of storytelling to embrace our shared humanity and duty to truth through a compassionate lens," the page reads.', 'The Archewell Audio page features the first episode of Archewell Audio, Harry and Meghan\'s new podcast with Spotify, as well as a short description of the production c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  after removing the cwd from sys.path.


# TF-IDF

In [ ]:
files = os.listdir('/content/drive/MyDrive/IR/Txt')
count = 0
list1=[]
for file in files:
       if count < 1000:
         count +=1
         list1.append(file)
       else:
          break
print(list1)
print(len(list1))

['162716.txt', '48025.txt', '135632.txt', '137490.txt', '80841.txt', '159191.txt', '123427.txt', '261483.txt', '110922.txt', '193662.txt', '24787.txt', '230731.txt', '133842.txt', '109579.txt', '46860.txt', '136236.txt', '189710.txt', '267787.txt', '30010.txt', '81830.txt', '20142.txt', '57744.txt', '81324.txt', '154676.txt', '264341.txt', '9405.txt', '93056.txt', '218183.txt', '295757.txt', '57361.txt', '111268.txt', '91411.txt', '85898.txt', '22347.txt', '271121.txt', '294520.txt', '151071.txt', '160039.txt', '129905.txt', '202567.txt', '9373.txt', '13430.txt', '23812.txt', '153240.txt', '133379.txt', '149833.txt', '210114.txt', '129570.txt', '195513.txt', '121103.txt', '122528.txt', '130313.txt', '217281.txt', '174146.txt', '40088.txt', '124982.txt', '34858.txt', '30516.txt', '117577.txt', '1028.txt', '22954.txt', '68837.txt', '133235.txt', '128974.txt', '34945.txt', '30333.txt', '93264.txt', '123007.txt', '135350.txt', '79545.txt', '287529.txt', '66859.txt', '186073.txt', '114325.t

In [ ]:
list1=['162716.txt', '48025.txt', '135632.txt', '137490.txt', '80841.txt', '159191.txt', '123427.txt', '261483.txt', '110922.txt', '193662.txt', '24787.txt', '230731.txt', '133842.txt', '109579.txt', '46860.txt', '136236.txt', '189710.txt', '267787.txt', '30010.txt', '81830.txt', '20142.txt', '57744.txt', '81324.txt', '154676.txt', '264341.txt', '9405.txt', '93056.txt', '218183.txt', '295757.txt', '57361.txt', '111268.txt', '91411.txt', '85898.txt', '22347.txt', '271121.txt', '294520.txt', '151071.txt', '160039.txt', '129905.txt', '202567.txt', '9373.txt', '13430.txt', '23812.txt', '153240.txt', '133379.txt', '149833.txt', '210114.txt', '129570.txt', '195513.txt', '121103.txt', '122528.txt', '130313.txt', '217281.txt', '174146.txt', '40088.txt', '124982.txt', '34858.txt', '30516.txt', '117577.txt', '1028.txt', '22954.txt', '68837.txt', '133235.txt', '128974.txt', '34945.txt', '30333.txt', '93264.txt', '123007.txt', '135350.txt', '79545.txt', '287529.txt', '66859.txt', '186073.txt', '114325.txt', '35909.txt', '182809.txt', '209068.txt', '128278.txt', '118116.txt', '77363.txt', '118400.txt', '122386.txt', '107829.txt', '52863.txt', '131390.txt', '129980.txt', '57064.txt', '166152.txt', '19864.txt', '295998.txt', '143885.txt', '77032.txt', '170978.txt', '161884.txt', '113008.txt', '95661.txt', '16779.txt', '150302.txt', '31569.txt', '301236.txt', '40528.txt', '132456.txt', '220902.txt', '253092.txt', '267803.txt', '167434.txt', '24893.txt', '53227.txt', '58888.txt', '190177.txt', '235414.txt', '166075.txt', '112736.txt', '109990.txt', '94685.txt', '232420.txt', '54176.txt', '211933.txt', '116180.txt', '143338.txt', '124548.txt', '226146.txt', '51812.txt', '234819.txt', '208639.txt', '170157.txt', '178730.txt', '192605.txt', '37229.txt', '105606.txt', '136398.txt', '78031.txt', '61610.txt', '165923.txt', '49369.txt', '117672.txt', '144298.txt', '38849.txt', '132007.txt', '112323.txt', '131128.txt', '119988.txt', '16429.txt', '904.txt', '128955.txt', '28435.txt', '4213.txt', '677.txt', '90451.txt', '108492.txt', '6337.txt', '187884.txt', '123747.txt', '137718.txt', '133503.txt', '35224.txt', '36058.txt', '138121.txt', '56232.txt', '52940.txt', '120934.txt', '34422.txt', '84627.txt', '73128.txt', '132772.txt', '128994.txt', '15261.txt', '17121.txt', '173301.txt', '156569.txt', '259794.txt', '8632.txt', '130297.txt', '21976.txt', '29551.txt', '123829.txt', '93537.txt', '219572.txt', '125374.txt', '181137.txt', '236301.txt', '118024.txt', '109155.txt', '115803.txt', '300162.txt', '178273.txt', '171800.txt', '171473.txt', '135202.txt', '101776.txt', '166939.txt', '187812.txt', '259056.txt', '64477.txt', '9419.txt', '145928.txt', '39882.txt', '149696.txt', '119619.txt', '4727.txt', '116808.txt', '249556.txt', '139534.txt', '39062.txt', '59666.txt', '91013.txt', '268078.txt', '135359.txt', '17990.txt', '78992.txt', '49966.txt', '108757.txt', '44942.txt', '94829.txt', '68803.txt', '157720.txt', '133025.txt', '89691.txt', '163628.txt', '215193.txt', '122379.txt', '259109.txt', '153771.txt', '19031.txt', '190224.txt', '174009.txt', '156148.txt', '82767.txt', '135453.txt', '191248.txt', '82253.txt', '101786.txt', '23629.txt', '119124.txt', '134614.txt', '151591.txt', '83109.txt', '132883.txt', '1662.txt', '51332.txt', '46861.txt', '113947.txt', '118909.txt', '9979.txt', '42183.txt', '259841.txt', '191123.txt', '206018.txt', '179332.txt', '134259.txt', '168921.txt', '244279.txt', '20703.txt', '52381.txt', '139909.txt', '1373.txt', '227715.txt', '67717.txt', '223861.txt', '72329.txt', '10422.txt', '108561.txt', '194613.txt', '209368.txt', '117475.txt', '185207.txt', '123152.txt', '220007.txt', '112046.txt', '261273.txt', '214980.txt', '163434.txt', '96953.txt', '32699.txt', '225944.txt', '161093.txt', '48698.txt', '158765.txt', '112123.txt', '233934.txt', '44351.txt', '247034.txt', '138981.txt', '198147.txt', '37892.txt', '225743.txt', '116505.txt', '37072.txt', '159387.txt', '194428.txt', '21937.txt', '59160.txt', '18606.txt', '119298.txt', '196141.txt', '292920.txt', '155850.txt', '274832.txt', '235227.txt', '91761.txt', '270044.txt', '135322.txt', '197294.txt', '253335.txt', '240444.txt', '130970.txt', '143082.txt', '22997.txt', '299086.txt', '262067.txt', '201370.txt', '228309.txt', '111947.txt', '1143.txt', '208642.txt', '18338.txt', '3003.txt', '111981.txt', '262172.txt', '236591.txt', '1214.txt', '137522.txt', '113539.txt', '129780.txt', '8065.txt', '255801.txt', '91812.txt', '210820.txt', '160821.txt', '48480.txt', '59483.txt', '47525.txt', '23920.txt', '84139.txt', '166029.txt', '211404.txt', '158145.txt', '121987.txt', '112296.txt', '12458.txt', '123570.txt', '230285.txt', '10798.txt', '134601.txt', '127237.txt', '260445.txt', '210170.txt', '121352.txt', '87111.txt', '130525.txt', '30467.txt', '36915.txt', '76222.txt', '116115.txt', '50504.txt', '76301.txt', '284930.txt', '118919.txt', '255931.txt', '17886.txt', '137777.txt', '24826.txt', '122587.txt', '260739.txt', '114843.txt', '142690.txt', '154593.txt', '114769.txt', '151618.txt', '5510.txt', '134818.txt', '178831.txt', '91561.txt', '41630.txt', '174862.txt', '25123.txt', '83201.txt', '289710.txt', '115277.txt', '188488.txt', '117481.txt', '126144.txt', '39762.txt', '274590.txt', '128825.txt', '144834.txt', '131746.txt', '293350.txt', '136933.txt', '199533.txt', '250831.txt', '139736.txt', '36526.txt', '134824.txt', '150806.txt', '84958.txt', '191335.txt', '273909.txt', '136568.txt', '134459.txt', '35025.txt', '115987.txt', '211842.txt', '125009.txt', '123376.txt', '195351.txt', '149362.txt', '290967.txt', '80888.txt', '118084.txt', '145857.txt', '174853.txt', '146704.txt', '10680.txt', '100615.txt', '97177.txt', '23959.txt', '216855.txt', '295712.txt', '186465.txt', '26756.txt', '142259.txt', '215354.txt', '70977.txt', '148809.txt', '121982.txt', '36164.txt', '95622.txt', '32610.txt', '87446.txt', '55054.txt', '255971.txt', '126633.txt', '109314.txt', '133542.txt', '206465.txt', '121799.txt', '284262.txt', '243441.txt', '126079.txt', '51489.txt', '43168.txt', '259740.txt', '192629.txt', '70091.txt', '155953.txt', '51963.txt', '103265.txt', '18562.txt', '96237.txt', '57957.txt', '147213.txt', '111637.txt', '185449.txt', '143441.txt', '260109.txt', '37662.txt', '98597.txt', '224011.txt', '233624.txt', '262552.txt', '70730.txt', '110124.txt', '109203.txt', '274932.txt', '135047.txt', '116488.txt', '300465.txt', '138047.txt', '184034.txt', '108646.txt', '183631.txt', '107979.txt', '18737.txt', '163282.txt', '108395.txt', '191078.txt', '131202.txt', '251243.txt', '88384.txt', '146339.txt', '231444.txt', '159856.txt', '290808.txt', '190409.txt', '230665.txt', '182432.txt', '148069.txt', '10517.txt', '69893.txt', '202618.txt', '139838.txt', '55363.txt', '35866.txt', '145955.txt', '223208.txt', '168889.txt', '205324.txt', '42559.txt', '109316.txt', '91472.txt', '295411.txt', '111760.txt', '181379.txt', '179325.txt', '164603.txt', '295569.txt', '16196.txt', '265778.txt', '168706.txt', '291072.txt', '255798.txt', '130496.txt', '246280.txt', '119578.txt', '75629.txt', '37708.txt', '130732.txt', '18214.txt', '109541.txt', '241814.txt', '130444.txt', '133074.txt', '80248.txt', '61702.txt', '68241.txt', '80664.txt', '24630.txt', '114977.txt', '136269.txt', '56617.txt', '216426.txt', '292933.txt', '63229.txt', '121580.txt', '58909.txt', '209935.txt', '153690.txt', '174956.txt', '198852.txt', '54749.txt', '125141.txt', '65732.txt', '49434.txt', '15839.txt', '165082.txt', '14131.txt', '72650.txt', '201995.txt', '225675.txt', '71996.txt', '292702.txt', '134321.txt', '191875.txt', '224649.txt', '170464.txt', '193208.txt', '152763.txt', '55791.txt', '31600.txt', '4188.txt', '193169.txt', '242065.txt', '236874.txt', '239738.txt', '47674.txt', '243723.txt', '124784.txt', '239475.txt', '260272.txt', '59052.txt', '27037.txt', '53131.txt', '105284.txt', '238594.txt', '166688.txt', '82804.txt', '155667.txt', '298318.txt', '30594.txt', '91208.txt', '39834.txt', '80789.txt', '52747.txt', '81511.txt', '62155.txt', '170671.txt', '64333.txt', '112926.txt', '86824.txt', '108075.txt', '116236.txt', '204680.txt', '96690.txt', '132903.txt', '142734.txt', '256620.txt', '132428.txt', '182049.txt', '22444.txt', '225657.txt', '11017.txt', '156385.txt', '267422.txt', '138140.txt', '1490.txt', '104768.txt', '221968.txt', '2573.txt', '166394.txt', '18172.txt', '111395.txt', '43601.txt', '150440.txt', '131612.txt', '145096.txt', '22110.txt', '126593.txt', '154015.txt', '144093.txt', '84397.txt', '36437.txt', '112906.txt', '27772.txt', '226226.txt', '59285.txt', '190020.txt', '95279.txt', '201970.txt', '245700.txt', '100077.txt', '27462.txt', '221697.txt', '139542.txt', '187635.txt', '6949.txt', '63091.txt', '233763.txt', '169626.txt', '132050.txt', '1416.txt', '178555.txt', '95537.txt', '129455.txt', '56507.txt', '109374.txt', '115356.txt', '154851.txt', '221883.txt', '282641.txt', '91002.txt', '194398.txt', '166314.txt', '229666.txt', '109496.txt', '155349.txt', '143253.txt', '289755.txt', '207560.txt', '13259.txt', '46721.txt', '269795.txt', '225389.txt', '55309.txt', '76393.txt', '154150.txt', '116904.txt', '47792.txt', '255777.txt', '108184.txt', '35324.txt', '81608.txt', '216869.txt', '46482.txt', '241646.txt', '167288.txt', '121497.txt', '117972.txt', '142537.txt', '159350.txt', '275002.txt', '34583.txt', '114874.txt', '17199.txt', '204501.txt', '21731.txt', '196828.txt', '127373.txt', '47910.txt', '95782.txt', '131388.txt', '14115.txt', '133832.txt', '23291.txt', '108077.txt', '111605.txt', '11615.txt', '132243.txt', '119689.txt', '41115.txt', '121689.txt', '3237.txt', '157597.txt', '181041.txt', '151213.txt', '106618.txt', '247960.txt', '21819.txt', '225200.txt', '135983.txt', '228348.txt', '218627.txt', '291972.txt', '83779.txt', '10055.txt', '252299.txt', '229521.txt', '174640.txt', '4652.txt', '108651.txt', '261135.txt', '795.txt', '136334.txt', '177853.txt', '185398.txt', '19029.txt', '154674.txt', '267212.txt', '214644.txt', '62592.txt', '146524.txt', '134379.txt', '263101.txt', '218161.txt', '91231.txt', '86187.txt', '69203.txt', '123529.txt', '72725.txt', '131901.txt', '117992.txt', '147362.txt', '91356.txt', '272958.txt', '52350.txt', '36399.txt', '4446.txt', '7232.txt', '9820.txt', '106965.txt', '196933.txt', '134650.txt', '109803.txt', '136719.txt', '200662.txt', '111534.txt', '41124.txt', '13614.txt', '99226.txt', '159127.txt', '21930.txt', '116766.txt', '135551.txt', '46696.txt', '119725.txt', '95541.txt', '57832.txt', '143516.txt', '106641.txt', '221600.txt', '80268.txt', '257063.txt', '294178.txt', '18285.txt', '251350.txt', '113261.txt', '194149.txt', '255787.txt', '187102.txt', '33550.txt', '189092.txt', '288328.txt', '142915.txt', '298401.txt', '114976.txt', '297201.txt', '140190.txt', '213306.txt', '198432.txt', '111556.txt', '108833.txt', '72383.txt', '25671.txt', '265943.txt', '183570.txt', '71470.txt', '27022.txt', '144317.txt', '39413.txt', '34385.txt', '62737.txt', '79585.txt', '35399.txt', '35093.txt', '10742.txt', '33790.txt', '172071.txt', '127137.txt', '21345.txt', '41577.txt', '258651.txt', '133852.txt', '76311.txt', '16985.txt', '217785.txt', '140988.txt', '124571.txt', '221628.txt', '69795.txt', '182108.txt', '247388.txt', '72857.txt', '124138.txt', '108222.txt', '23024.txt', '239086.txt', '148879.txt', '125356.txt', '187189.txt', '208233.txt', '15034.txt', '241982.txt', '7808.txt', '157529.txt', '41620.txt', '12635.txt', '120240.txt', '145827.txt', '258698.txt', '52544.txt', '178045.txt', '145408.txt', '17123.txt', '236986.txt', '229168.txt', '265656.txt', '112723.txt', '26997.txt', '106665.txt', '19848.txt', '143285.txt', '260522.txt', '12616.txt', '138850.txt', '9815.txt', '33105.txt', '105690.txt', '197549.txt', '54029.txt', '134067.txt', '35894.txt', '32259.txt', '240971.txt', '118130.txt', '239716.txt', '22611.txt', '142842.txt', '76634.txt', '273557.txt', '147697.txt', '178410.txt', '147853.txt', '41016.txt', '137287.txt', '127466.txt', '255735.txt', '14617.txt', '82867.txt', '37895.txt', '176188.txt', '197204.txt', '198011.txt', '43760.txt', '580.txt', '140467.txt', '280636.txt', '133627.txt', '288442.txt', '236912.txt', '172034.txt', '275163.txt', '159416.txt', '43040.txt', '262703.txt', '176172.txt', '201196.txt', '83943.txt', '18110.txt', '51190.txt', '164557.txt', '186639.txt', '96019.txt', '181631.txt', '55857.txt', '59400.txt', '244879.txt', '182427.txt', '282250.txt', '99679.txt', '169594.txt', '138734.txt', '136371.txt', '61516.txt', '126804.txt', '54244.txt', '171882.txt', '153161.txt', '23809.txt', '73415.txt', '164123.txt', '135361.txt', '136324.txt', '236714.txt', '8551.txt', '286351.txt', '111272.txt', '259735.txt', '62611.txt', '111045.txt', '215253.txt', '169696.txt', '148475.txt', '136385.txt', '77825.txt', '115759.txt', '136026.txt', '85696.txt', '104629.txt', '128947.txt', '282460.txt', '259812.txt', '126242.txt', '247372.txt', '51258.txt', '192304.txt', '300257.txt', '185222.txt', '228998.txt', '31080.txt', '249154.txt', '26494.txt', '148905.txt', '132828.txt', '106358.txt', '184809.txt', '151786.txt', '53042.txt', '51402.txt', '109692.txt', '28782.txt', '42182.txt', '22149.txt', '146181.txt', '54023.txt', '222869.txt', '119863.txt', '186880.txt', '58382.txt', '114810.txt', '92160.txt', '164858.txt', '226673.txt', '167357.txt', '30575.txt', '224815.txt', '189233.txt', '188071.txt', '157857.txt', '199145.txt', '48947.txt', '55058.txt', '199103.txt', '186208.txt', '131045.txt', '67537.txt', '69369.txt', '291312.txt', '138502.txt', '20356.txt', '19594.txt', '89765.txt', '110004.txt', '118004.txt', '200405.txt', '38393.txt', '106984.txt', '4565.txt', '16217.txt', '33221.txt', '69845.txt']

In [ ]:
pg=requests.get('https://www.townandcountrymag.com/society/tradition/a35104292/meghan-markle-prince-harry-princess-diana-archewell-website-update/')
wiki = bs4.BeautifulSoup(pg.text, "lxml")
x=[i.getText() for i in wiki.select('p')]

In [ ]:
files = os.listdir('/content/drive/MyDrive/IR/Txt')
count = 0
list1=[]
for file in files:
       if count < 1000:
         count +=1
         list1.append(file)
       else:
          break
print(list1)
print(len(list1))

['162716.txt', '48025.txt', '135632.txt', '137490.txt', '80841.txt', '159191.txt', '123427.txt', '261483.txt', '110922.txt', '193662.txt', '24787.txt', '230731.txt', '133842.txt', '109579.txt', '46860.txt', '136236.txt', '189710.txt', '267787.txt', '30010.txt', '81830.txt', '20142.txt', '57744.txt', '81324.txt', '154676.txt', '264341.txt', '9405.txt', '93056.txt', '218183.txt', '295757.txt', '57361.txt', '111268.txt', '91411.txt', '85898.txt', '22347.txt', '271121.txt', '294520.txt', '151071.txt', '160039.txt', '129905.txt', '202567.txt', '9373.txt', '13430.txt', '23812.txt', '153240.txt', '133379.txt', '149833.txt', '210114.txt', '129570.txt', '195513.txt', '121103.txt', '122528.txt', '130313.txt', '217281.txt', '174146.txt', '40088.txt', '124982.txt', '34858.txt', '30516.txt', '117577.txt', '1028.txt', '22954.txt', '68837.txt', '133235.txt', '128974.txt', '34945.txt', '30333.txt', '93264.txt', '123007.txt', '135350.txt', '79545.txt', '287529.txt', '66859.txt', '186073.txt', '114325.t

In [ ]:
with open('/content/drive/MyDrive/IR/result.txt', 'w') as outfile:
    for i in list1:
      fname = "/content/drive/MyDrive/IR/Txt/"+str(i)
      with open(fname,'r') as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
corp=[]
for i in list1:
      fname = "/content/drive/MyDrive/IR/Txt/"+str(i)
      with open(fname,'r') as infile:
            cor=[]
            for line in infile:
                cor.append(line)
            corp.append(cor)              

In [ ]:
cleanCorp=[]
for i in corp:
   s=""
   for lines in i:
      s=s+lines
   cleanCorp.append(cleaning(s))

In [ ]:
file = open('/content/drive/MyDrive/IR/result.txt','r')
file = file.read()
clean1=cleaning(file)

In [ ]:
cleanCorp1=[]
for i in cleanCorp:
   s=""
   for lines in i:
      s=s+lines
   cleanCorp1.append(sclean(s))

In [ ]:
joblib.dump(cleanCorp1,'/content/drive/My Drive/IR/Project_corp1000')

['/content/drive/My Drive/IR/Project_corp1000']

In [ ]:
c=joblib.load('/content/drive/MyDrive/IR/Project_Text_Clean')

In [ ]:
cleanCorp1=joblib.load('/content/drive/MyDrive/IR/Project_corp1000')

In [ ]:
dict1={'pageid': list1, 'text': cleanCorp1}
df = pd.DataFrame(dict1)
df['text'] = df['text'].astype('str')

In [ ]:
joblib.dump(df,'/content/drive/MyDrive/IR/df.csv')

['/content/drive/MyDrive/IR/df.csv']

In [ ]:
vectoriser = TfidfVectorizer(ngram_range=(1,1), max_features=5000)
vectoriser.fit(df['text'])
print('Vectoriser fitted.')
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

Vectoriser fitted.
No. of feature_words:  5000


In [ ]:
listfinal=[]
for i in x:
  listx = []
  listx.append(i)
  r=vectoriser.transform(listx)
  r=r.toarray()
  listfinal.append(r[0])
listfinal

In [ ]:
query='an American public organization that focuses on non-profit activities and creative media ventures.[1][2]'
qlist=[]
qlist.append(query)
qr=vectoriser.transform(qlist)
qr=qr.toarray()

In [ ]:
scores= []
for i in range(len(listfinal)):
  scores.append(distance.cosine(listfinal[i],qr[0]))
def ranking(X,Y):
  #X = X.split('\n\n') 
  score_for_each_para = Y
  return [x for _,x in sorted(zip(score_for_each_para,X))]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
rank = ranking(x,scores)
print(rank[0])
print(rank[1])

Just ahead of the new year, Prince Harry and Meghan Markle have updated the website for their new non-profit, Archewell. The site was launched in October with just a landing page, but now, the couple has fleshed it out, adding pages for the Archewell Foundation, Archewell Audio, and Archewell Productions.
"I am my mother’s son. And I am our son’s mother. Together we bring you Archewell," the couple wrote on the homepage for their new non-profit.


# For internal Links -

# Find page id with page title

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IR/Project/page.csv')

In [ ]:
def find_pageid_with_name(s,data):
 for i in range(len(data)):
  if data.iloc[i]['title'] == s:
    return data.iloc[i]['page_id']
 return None

In [ ]:
pageid=find_pageid_with_name('Babylonian Chronicles',data) #pass csv and title of page here
pageid#currentpage where query offset will be found

1895473

# Find page with page id

In [ ]:
def search(target_id):
   NUM_KLAT_LINES = 5_343_564  
   with open("/content/drive/MyDrive/IR/Project/link_annotated_text.jsonl") as fp:
      for line in fp:
        page = json.loads(line)
        if page['page_id']==target_id:
          print(page['page_id'])
          x = len(page['sections'])
          i=0
          while i<x:
            print("name: "+str(page['sections'][i]['name']))
            print("text: "+str(page['sections'][i]['text']))
            print("link_offset"+str(page['sections'][i]["link_offsets"]))
            print(page['sections'][i]["link_lengths"])
            print(page['sections'][i]["target_page_ids"])
            i=i+1
          return page
          break
      return None

In [ ]:
if pageid != None:
 page=search(pageid)

1895473
name: Introduction
text: The Babylonian Chronicles are a series of tablets recording major events in Babylonian history. They are thus one of the first steps in the development of ancient historiography. The Babylonian Chronicles were written from the reign of Nabonassar up to the Parthian Period, by Babylonian astronomers ("Chaldaeans"), who probably used the Astronomical Diaries as their source. Almost all of the tablets were identified as chronicles once in the collection of the British Museum, having been acquired via antiquities dealers from unknown excavations in the 19th century. All but three of the chronicles are unprovenanced. The Chronicles provide the "master narrative" for large tracts of current Babylonian history.
link_offset[42, 76, 155, 163, 236, 257, 338, 462, 605]
[6, 7, 7, 14, 10, 15, 20, 14, 13]
[306575, 20609622, 51320, 13276, 159121, 4501200, 37579426, 4675, 288400]
name: Discovery and publication
text: The chronicles are thought to have been transferred 

# Query check from target id

In [ ]:
# Flow
# from wensite system
# PageTitle#from website
# CurrentPageId = find_pageid_with_name(PageTitle,Page.csv)
# PageObj = search(CurrentPageId)
# TargetId = searchQueryRettarget(Query,PageObj)
# path = searchTargetRetTXT(TargetId)

In [ ]:
def searchTargetRetTXT(target_id):
   NUM_KLAT_LINES = 5_343_564  
   with open("/content/drive/MyDrive/IR/Project/link_annotated_text.jsonl") as fp:
      for line in fp:
        page = json.loads(line)
        if page['page_id']==target_id:
          x = len(page['sections'])
          i=0
          f= open("/content/drive/MyDrive/IR/Project/"+str(page['page_id'])+".txt", "a")
          stringTopaste=""
          while i<x:#append documents for sending to the similarity check as a txt
            stringTopaste= stringTopaste + str(page['sections'][i]['name'])+"\n"+str(page['sections'][i]['text'])+"\n"
            i=i+1
          print(stringTopaste)  
          f.write(stringTopaste)
          f.close()
          return "/content/drive/MyDrive/IR/Project/"+str(page['page_id'])+".txt"
          break
      return None

In [ ]:
def searchQueryRetTarget(Query,PageObj):
  length = len(PageObj['sections'])
  i = -1
  QueryNotmatch = False
  start =None
  end = None
  wewantlist=[]
  positionofstart = 0
  positionofend = 0
  while i < length:
    i=i+1
    sectionText = PageObj['sections'][i]['text']
    if Query in sectionText:
      splitedQuery = Query.split(" ")
      splittedText = sectionText.split(" ")
      j=-1
      for word in splittedText:
        j=j+1
        positionofstart = positionofstart + len([char for char in word])
        if word == splitedQuery[0]:
          start = j
          k = j
          for q in splitedQuery:
            if splittedText[k]!=q:
              QueryNotmatch =True
              break
            k = k+1
          if QueryNotmatch == False:
            end = j+len(splitedQuery)
            positionofend = positionofstart + len([char for char in Query]) - len([char for char in splitedQuery[0]])
            break
        positionofstart = positionofstart + 1
  #check offset
      if start !=None: 
        for counter in range(len(PageObj['sections'][i]['link_offsets'])):
          if positionofstart <= PageObj['sections'][i]['link_offsets'][counter] and positionofend >= PageObj['sections'][i]['link_offsets'][counter]:
            wewantlist.append(PageObj['sections'][i]['target_page_ids'][counter])
      break
  return wewantlist

In [ ]:
page = search(1895473)
string="The Babylonian Chronicles are a series of tablets recording major events in Babylonian history."
wewantlist = searchQueryRetTarget(string,page)
print(wewantlist)

1895473
name: Introduction
text: The Babylonian Chronicles are a series of tablets recording major events in Babylonian history. They are thus one of the first steps in the development of ancient historiography. The Babylonian Chronicles were written from the reign of Nabonassar up to the Parthian Period, by Babylonian astronomers ("Chaldaeans"), who probably used the Astronomical Diaries as their source. Almost all of the tablets were identified as chronicles once in the collection of the British Museum, having been acquired via antiquities dealers from unknown excavations in the 19th century. All but three of the chronicles are unprovenanced. The Chronicles provide the "master narrative" for large tracts of current Babylonian history.
link_offset[42, 76, 155, 163, 236, 257, 338, 462, 605]
[6, 7, 7, 14, 10, 15, 20, 14, 13]
[306575, 20609622, 51320, 13276, 159121, 4501200, 37579426, 4675, 288400]
name: Discovery and publication
text: The chronicles are thought to have been transferred 